In [154]:
import pandas as pd
data = pd.read_csv("Tripadvisordata_red.csv")

data= data.round(2)

list = []
for i in range(len(data["Type of food"])):  
    if "<div" in str(data["Type of food"][i]):
        list.append(i)
    
#print(list)
for i in list:
    data["Type of food"][i] = "Missing"


# set color 
green_color = "#00AF87"

C:\Users\sofie\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [155]:
print(set(data["Type of food"]))


{'Centraleuropæisk', nan, 'Europæisk', 'Delikatesseforretning', 'Bar', 'Skandinavisk', 'Brasiliansk', 'Vietnamesisk', 'Gademad', 'Arabisk', 'Egyptisk', 'Tyrkisk', 'Nepalesisk', 'Dansk', 'Caribisk', 'Asiatisk', 'Libanesisk', 'Ølpub', 'Belgisk', 'Missing', 'Sund', 'Grill', 'Moderne', 'Fisk og skaldyr', 'Sushi', 'Fusion', 'Mexicansk', 'Fransk', 'Græsk', 'New Zealand', 'Fastfood', 'Centralitaliensk', 'Vinstue', 'Indisk', 'Kinesisk', 'Pizza', 'Steakhouse', 'Indonesisk', 'Tysk', 'International', 'Middelhavsområdet', 'Spisested', 'Thai', 'Café', 'Marokkansk', 'Italiensk', 'Spansk', 'Mellemøstlig', 'Afrikansk', 'Hawaiiansk', 'Argentinsk', 'Amerikansk', 'Japansk', 'Latinamerikansk'}


In [156]:
sum_tab = data.describe()
sum_tab.to_csv("Summary_tab.csv")

In [157]:
import matplotlib.pyplot as plt

mod_df = data 

# Obs filter restaurants more than 15 km from Kgs. Nytorv
mod_df = mod_df.drop(mod_df[mod_df['Distance from Kgs. Nytorv'] > 15000 ].index , inplace=True)

#mod_df.describe()
# OBS kør den forfra hvis du vil ændre i noget. 


In [158]:
# Mean ratings grouped by type of food
bar_df = mod_df.groupby(['Type of food'])['Main rating'].mean()
bar_df = pd.DataFrame(bar_df)

#bar_df.columns = ['Type of food', "Mean rating"]
bar_df2 = mod_df.groupby(['Type of food'])['Main rating'].count()
bar_df2 = pd.DataFrame(bar_df2)

bar_df3 = pd.merge(bar_df, bar_df2, on='Type of food', how='left')
bar_df3 = pd.DataFrame(bar_df3)

bar_df3.columns = ["Mean rating", "Number of restaurants"]
bar_df3.reset_index(inplace=True)

# Only if more than 10 restaurants
bar_df4 = bar_df3.loc[bar_df3['Number of restaurants'] > 10] 
bar_df4 = bar_df4.loc[bar_df3['Type of food'] != "Missing"] 

bar_df4 = bar_df4.sort_values(by=["Mean rating"])
bar_df4

AttributeError: 'NoneType' object has no attribute 'groupby'

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcdefaults()
fig, ax = plt.subplots()
plt.style.use('ggplot')

# Example data
rests = [bar_df4["Mean rating"]]
y_pos = bar_df4["Type of food"]
performance = bar_df4["Mean rating"]

ax.barh(y_pos, performance, align='center', color = green_color)
#ax.set_yticks(y_pos)
#ax.set_yticklabels("Type of food")
#ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Mean rating')
ax.set_xlim(3,5)
#ax.set_title('How fast do you want to go today?')
label = [bar_df4["Mean rating"]]




In [ ]:
%matplotlib inline
plt.style.use('seaborn')

mod_df["Reverse ranking"] = mod_df["Ranking on list"]*(-1)

mod_df.plot(x='Distance from Kgs. Nytorv', y='Reverse ranking', kind='scatter')
plt.show()

In [ ]:

%matplotlib inline
plt.style.use('seaborn')

#mod_df.plot(x='Distance from Kgs. Nytorv', y='Service', kind='scatter')
#plt.show()